# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set Interface Parameters (e.g. `distance_z`, `max_area`, `miller_indices`) below or use the default values
3. Additionally, specify whether termination is selected using interactive prompt, or the via the variable in the code.
3. Click "Run" > "Run All Cells" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## 1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` as directed in README.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

In [29]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

from utils.visualize import visualize_materials as visualize

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[0]
film = materials[1]
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

Data from 0-Ni has been read successfully.
Data from 1-Graphene has been read successfully.
Data from 3-WS2 has been read successfully.
Data from 4-BN has been read successfully.
Data from 4-Te2Mo has been read successfully.
Data from Ni12 C8, Interface, Strain:0.105% has been read successfully.


GridBox(children=(VBox(children=(Label(value='Ni - Material 0', layout=Layout(align_self='center', height='30p…

### 2.1. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
substrate_slab = Slab(
    bulk=substrate,
    # the Miller indices of the interfacial plane of the substrate
    miller_indices=(1, 1, 1),
    # substrate thickness in layers
    thickness=3,
    # the matrix of the surface cell along the basal plane dimension
    # xy_supercell_matrix=[[1,0], [0,1]],
)

film_slab = Slab(
    bulk=film,
    # the Miller indices of the interfacial plane of the layer
    miller_indices=(0, 0, 1),
    # layer thickness in layers
    thickness=1,
    # the matrix of the surface cell along the basal plane dimension
    # xy_supercell_matrix=[[1,0], [0,1]],
)

In [ ]:
visualize([substrate_slab, film_slab], repetitions=[3, 3, 1], rotation="0x")

## 2. Set Interface Parameters

### 2.1. Set Substrate and Layer Parameters 
Imported `InterfaceBuilderSettings` is a class that specifies the parameters for the builder that generates interfaces. The default values are assumed if properties are not set during the initialization.
Additionally, specify if the termination selection is done using interactive prompt, or the via selecting the termination index in the code.

In [25]:

# Parameters can be set during the class initialization:
builder_with_no_strain_matching = InterfaceBuilder(
    substrate_slab=substrate_slab,
    film_parameters=film_slab,
    # distance between substrate and layer, in Angstroms
    distance_z=3.0,
    # shift in x direction, in Angstroms
    shift_x=0.0,
    # shift in y direction, in Angstroms
    shift_y=0.0,
    # if True, the surface plane is constructed using miller indices of the conventional cell
    use_conventional_cell=True,
    use_strain_matching=False,
    # strain_matching_algorithm=StrainMatchingType.ZSL,
    # strain_matching_parameters=ZSLStrainMatchingParameters()
)

IS_TERMINATION_SELECTION_INTERACTIVE = False  # if True, the user can select the termination interactively
TERMINATION_INDEX = 0  # the default termination index that is used if no termination selected, ignored in interactive mode

ImportError: cannot import name 'ZSLStrainMatchingParameters' from 'mat3ra.made.tools.build.interface' (/Users/mat3ra/code/GREEN/api-examples/.venv/lib/python3.11/site-packages/mat3ra/made/tools/build/interface.py)

### 2.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

## 3. Load input Materials


## 4. Create interfaces

### 4.1. Initialize the interface builder

Initialize the interface builder with the materials and interface settings.

### 4.2. Select the termination
Possible terminations for the interface are found by the interface builder. The user can select the termination interactively or use the default one.

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide
terminations = interface_builder.terminations

if IS_TERMINATION_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        selected_termination = await ui_prompt_select_array_element_by_index_pyodide(terminations, element_name="termination")
    else:
         selected_termination = ui_prompt_select_array_element_by_index(terminations, element_name="termination")
else:    
    selected_termination = terminations[TERMINATION_INDEX]

### 4.3. Create interfaces for the selected termination

In [ ]:
from mat3ra.made.tools.build import create_interfaces

interface = create_interface(
    interface_builder=builder_with_no_strain_matching,
    termination=selected_termination,
)

### 4.4. Print out interface with the lowest strain for selected termination

In [ ]:
print(f"Interface for termination {selected_termination}:")

## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


## 6. Select the interface to pass outside this kernel

### 6.1. Select the interface with the desired termination and strain

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

### 6.2. Visualize the selected interface(s)

In [ ]:
visualize(interface, repetitions=[1, 1, 1], rotation="0x")

### 6.3. Pass data to the outside runtime
Enrich the selected interfaces names with the strain values and pass them to the application runtime.

In [ ]:
from utils.jupyterlite import set_data

for interface in selected_interfaces:
    if "Interface, Strain:" not in interface["name"]:
        interface["name"] = f'{interface["name"]}, Interface, Strain:{interface["metadata"]["interface_properties"]["mean_abs_strain"]*100:.3f}%'

set_data("materials", selected_interfaces)